<a href="https://colab.research.google.com/github/jchen8000/GenerativeAI/blob/main/6_Deployment/Chatbot_FLAN_T5_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.11 Chatbot, Example of LLM-Powered Application

## Chatbot built with local LLM, flan-t5-base plus LoRA.

Make sure flan-t5-base_lora model is in the output folder

In [ ]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers --quiet
!pip install peft --quiet
!pip install sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import GenerationConfig
from peft import PeftModel, PeftConfig

In [ ]:
# Load the FLAN-T5-Base model and tokenizer
base_model_name = 'google/flan-t5-base'
lora_model_name = 'outputs/flan-t5-base_lora'

def load_model(base_model_name, lora_model_name):
    tokenizer = T5Tokenizer.from_pretrained(base_model_name)
    base_model = T5ForConditionalGeneration.from_pretrained(base_model_name,
                                                       torch_dtype=torch.bfloat16)
    model = PeftModel.from_pretrained(base_model,
                                      lora_model_name,
                                      torch_dtype=torch.bfloat16,
                                      is_trainnable=False)

    return tokenizer, model

def generate_output(tokenizer, model, input_text, max_length=200):
    # Tokenize the input text and generate the model's output
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(max_new_tokens=200, num_beams=1) )

    # Decode the generated tokens to a string
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

tokenizer, model = load_model(base_model_name, lora_model_name)


total_parameter = sum(p.numel() for p in model.parameters())
print(f"Parameters of the model: {total_parameter:,}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Parameters of the model: 249,347,328


In [ ]:
def chatbot():

    print("Chatbot initialized. You can start chatting now (type 'quit' to stop)!\n")

    while True:
        # Get user input
        user_input = input("You: ")

        # Check if the user wants to quit
        if user_input.lower() == "quit":
            break

        response = generate_output(tokenizer, model, user_input)

        # Print the generated text
        print(f"Chatbot: {response}\n")

# Run the chatbot
chatbot()

Chatbot initialized. You can start chatting now (type 'quit' to stop)!

You: How are you doing today
Chatbot: You are doing well today.

You: Summarize: Tesco please bring security back to the Hall Green store. The store is getting a more an more uncomfortable vibe, not alone on this either! <BR> Hi there, sorry to be a pain but can you confirm which Hall Green store this is? TY Reece <BR> Tesco Its the Hall Green store right next to the train station. Hoping you havent removed security from the others too now. <BR> Hi, can you please confirm what you mean by uncomfortable vibe? Nick <BR> Tesco Well theres pretty obvious shop lifters regularly, and today we had a man clearly intoxicated screaming and randomly asking people things. <BR> Thanks for taking the time to provide these details, can you just confirm its the Express store? TYPhil <BR> Tesco Yes the express store! Thanks aswell. Id review the CCTV from when security were removed. If customers can see the changes you will too! <B